In [3]:
import sys, os
import numpy as np
from numpy.linalg import LinAlgError, inv
from scipy.linalg import sqrtm

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(r"C:\Users\super\DAG")
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
print(os.getcwd())

import torch
import torch.nn as nn
from torch.optim import Adam, LBFGS
from nodag_gumbel_softmax import train_gumbel_sgd
from SCM_data import generate_scm_data
import numpy as np
from numpy.linalg import LinAlgError, inv
from scipy.linalg import sqrtm
import MEC

c:\Users\super\DAG\nodag


In [4]:
!pip install gglasso
from gglasso.problem import glasso_problem


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [5]:
# test
n = 10000
W_true = np.array([[0, 1, 0], [0, 0, 0], [0, 1, 0]])
Omega_true = np.eye(3)
Theta_true = (np.eye(3) - W_true) @ inv(Omega_true) @ (np.eye(3) - W_true.T)
B_init = np.eye(3)
print("Theta_true\n", Theta_true)
Sigma_true = inv(Theta_true)
print("Sigma_true\n",Sigma_true)

B_final,G_final, info = train_gumbel_sgd(
        Rhat_np = Sigma_true,
        lam = 0.5,
        delta = 1e-6,
        B_init = B_init
    )

print("Final loss:", info["final_loss"])
print("B (final):\n", B_final)
print("G_final:\n", G_final)

Theta_true
 [[ 2. -1.  1.]
 [-1.  1. -1.]
 [ 1. -1.  2.]]
Sigma_true
 [[1. 1. 0.]
 [1. 3. 1.]
 [0. 1. 1.]]
First loss =  4.999994000003
First logdet =  -5.999996999508401e-06
Final loss: 4.098604824568995
B (final):
 [[0.9999995  0.         0.        ]
 [0.         0.57734977 0.        ]
 [0.         0.         0.9999995 ]]
G_final:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [8]:
# init: ground truth
# lam = 0.5, tau = 0.8 -> 0.2

times = 1
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = seed)
        A_true = (np.eye(3) - G_true)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        
        #print(data.T@ data / 10000)
        B_final,G_final, info = train_gumbel_sgd(
        Rhat_np = Sigma_true,
        lam = 0.5,
        delta = 1e-6,
        B_init = A_true
    )
        
        print("G_est = \n",G_final)
        if MEC.is_in_markov_equiv_class(G_true, G_final): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
    print(f"SCM {i} : {true_count[i-1]/times}")

First loss =  4.999994000003
First logdet =  -5.999996999508401e-06
G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  4.098604824568995
SCM 1 : 1.0
First loss =  4.499994000003
First logdet =  -5.999996999508401e-06
G_est = 
 [[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  4.1931403521348205
SCM 2 : 1.0
First loss =  3.9999940000030003
First logdet =  -5.999996999508401e-06
G_est = 
 [[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
Final Loss =  3.9999940000015006
SCM 3 : 1.0
First loss =  5.999994000003
First logdet =  -5.999996999508401e-06
G_est = 
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]
Final Loss =  4.287675611033328
SCM 4 : 1.0
First loss =  9.499994000003
First logdet =  -5.999996999508401e-06
G_est = 
 [[0. 1. 1.]
 [0. 0. 1.]
 [0. 0. 0.]]
Final Loss =  4.4999937573610636
SCM 5 : 1.0


In [105]:
# init: Cholesky
# lam = 0.5, tau = 0.8 -> 0.2

from sympy.printing.pretty.pretty_symbology import B


times = 1
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = seed)
        A_true = (np.eye(3) - G_true)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        Theta_hat = inv(R_hat)
        A_true = np.eye(3) - G_true
        likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
        
        #print(data.T@ data / 10000)
        B_final,G_final, info = train_gumbel_sgd(
        Rhat_np = R_hat,
        lam = 0.3,
        delta = 1e-6,
        max_steps = 5000,
        tau_end = 0.05,
        B_init = B_init
        )
            
        print("G_est = \n",G_final)
        if MEC.is_in_markov_equiv_class(G_true, G_final): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
        print("Final penalty = ", info["final_penalty"])
        print("Final likelihood = ", info["final_likelihood"])
        print("likelihood_true = ", likelihood_true)
    print(f"SCM {i} : {true_count[i-1]/times}")
    print("")

First loss =  7.412313476079614
First logdet =  3.4366599182055086
G_est = 
 [[0. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
Final Loss =  3.5557638264833695
Final penalty =  0.6
Final likelihood =  2.9557638264833694
likelihood_true =  2.9562272795925044
SCM 1 : 0.0

First loss =  8.346654114858541
First logdet =  3.4366599182055086
G_est = 
 [[-1.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0. -1.]]
Final Loss =  3.855743981198083
Final penalty =  0.8999999999999999
Final likelihood =  2.9557439811980832
likelihood_true =  2.956227279592505
SCM 2 : 0.0

First loss =  7.553109335441061
First logdet =  3.4366599182055086
G_est = 
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]
Final Loss =  3.8469670005960594
Final penalty =  0.6
Final likelihood =  3.2469670005960594
likelihood_true =  2.9562272795925035
SCM 3 : 0.0

First loss =  9.019302516995813
First logdet =  3.4366599182055086
G_est = 
 [[ 0.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  1. -1.]]
Final Loss =  4.1410485248035585
Final penalty =  0.8999999999999999
Final likeli

In [4]:
# init: random
# lam = 0.5, tau = 0.8 -> 0.05

times = 1
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = seed)
        A_true = (np.eye(3) - G_true)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        B_init = np.random.randn(*R_hat.shape)
        likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
        
        B_final,G_final, info = train_gumbel_sgd(
        Rhat_np = R_hat,
        lam = 1,
        delta = 1e-6,
        max_steps = 5000,
        tau_end = 0.05,
        B_init = B_init
        )
            
        print("G_est = \n",G_final)
        if MEC.is_in_markov_equiv_class(G_true, G_final): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
        print("Final penalty = ", info["final_penalty"])
        print("Final likelihood = ", info["final_likelihood"])
        print("likelihood_true = ", likelihood_true)
    print(f"SCM {i} : {true_count[i-1]/times}")

G_est = 
 [[-1.  1.  0.]
 [ 0. -1.  1.]
 [ 1.  0. -1.]]
Final Loss =  5.955743992022789
Final penalty =  3.0
Final likelihood =  2.9557439920227893
likelihood_true =  2.9562272795925044
SCM 1 : 0.0
G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  4.641383444061964
Final penalty =  1.0
Final likelihood =  3.6413834440619643
likelihood_true =  2.956227279592505
SCM 2 : 0.0
G_est = 
 [[ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  1. -1.]]
Final Loss =  6.350238921720871
Final penalty =  3.0
Final likelihood =  3.3502389217208712
likelihood_true =  2.9562272795925035
SCM 3 : 1.0
G_est = 
 [[ 0.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1. -1.]]
Final Loss =  6.635577366963171
Final penalty =  3.0
Final likelihood =  3.6355773669631706
likelihood_true =  2.956227279592504
SCM 4 : 0.0
G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 0.]]
Final Loss =  15.72868352035102
Final penalty =  2.0
Final likelihood =  13.72868352035102
likelihood_true =  2.956227279592534
SCM 5 : 0.0


In [116]:
# init: GL
# lam = 0.5, tau = 0.8 -> 0.05

from sympy.printing.pretty.pretty_symbology import B


times = 1
n = 10000
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i, n, seed = seed)
        A_true = (np.eye(3) - G_true)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        Theta_hat = inv(R_hat)
        P = glasso_problem(R_hat, n, reg_params = {'lambda1': 0.05}, latent = False, do_scaling = False)
        lambda1_range = np.logspace(0, -3, 30)
        modelselect_params = {'lambda1_range': lambda1_range}

        P.model_selection(modelselect_params = modelselect_params, method = 'eBIC', gamma = 0.1)
        B_init = P.solution.adjacency_ + np.eye(3) + 0.01 * np.random.randn(*R_hat.shape)
        if i == 5: G_true = np.array([[0,1,2],[0,0,3],[0,0,0]])
        A_true = np.eye(3) - G_true
        likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
        
        #print(data.T@ data / 10000)
        B_final,G_final, info = train_gumbel_sgd(
        Rhat_np = R_hat,
        lam = 0.5,
        delta = 1e-6,
        max_steps = 5000,
        tau_end = 0.05,
        B_init = B_init
        )
            
        print("G_est = \n",G_final)
        if MEC.is_in_markov_equiv_class(G_true, G_final): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
        print("Final penalty = ", info["final_penalty"])
        print("Final likelihood = ", info["final_likelihood"])
        print("likelihood_true = ", likelihood_true)
    print(f"SCM {i} : {true_count[i-1]/times}")
    print("")

ADMM terminated after 8 iterations with status: optimal.
ADMM terminated after 23 iterations with status: optimal.
ADMM terminated after 23 iterations with status: optimal.
ADMM terminated after 23 iterations with status: optimal.
ADMM terminated after 23 iterations with status: optimal.
ADMM terminated after 14 iterations with status: optimal.
ADMM terminated after 8 iterations with status: optimal.
ADMM terminated after 8 iterations with status: optimal.
ADMM terminated after 8 iterations with status: optimal.
First loss =  2.9562252135827363
First logdet =  -0.0025632093208648177
G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Final Loss =  2.955802894760634
Final penalty =  1.2677127182677778e-56
Final likelihood =  2.955802894760634
likelihood_true =  2.9562272795925044
SCM 1 : 1.0

ADMM terminated after 11 iterations with status: optimal.
ADMM terminated after 9 iterations with status: optimal.
ADMM terminated after 9 iterations with status: optimal.
ADMM terminated after 10 itera

In [ ]:
# ER1, d=4
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B

seed = 1

# Load dataset
n, d = 10000, 4
graph_type, degree = 'ER', 0.5    # ER2 graph
B_scale = 1.0
noise_type = 'gaussian_ev'

Dataset = SyntheticDataset(n, d, graph_type, degree,
                          noise_type, B_scale, seed=seed)
data = Dataset.X
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
A_init = np.random.randn(*R_hat.shape)
B_true = Dataset.B
G_true = Dataset.B_bin
print("A_init = \n", A_init)
beta = 5
lam = 0.5

A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("A_true = \n",A_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty
print("penalty_true = ", true_penalty)
print("f_true = ", f_true)

B_final,G_final, info = train_gumbel_sgd(
    Rhat_np = R_hat,
    lam = lam,
    delta = 1e-6,
    B_init = A_init
)
    
print("G_est = \n",G_final)
if MEC.is_in_markov_equiv_class(G_true, G_final): true_count[i-1] += 1
print("Final Loss = ", info["final_loss"])
print("Final penalty = ", info["final_penalty"])
print("Final likelihood = ", info["final_likelihood"])
print("likelihood_true = ", likelihood_true)


(4, 4)
A_init = 
 [[ 0.50942359  0.20267527 -0.84409431 -0.55759826]
 [ 0.38671801  0.92360906  0.24063005  0.99121596]
 [ 0.5896104   0.32470472 -0.06313458 -0.84722912]
 [ 0.47030369  0.56950252  0.13970669 -0.24872583]]
A_true = 
 [[ 1.         -1.29974793  1.78941959  1.70284777]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
likelihood_true =  3.99747608723802
penalty_true =  1.5
f_true =  5.49747608723802
First loss =  14.362069503263022
First logdet =  1.6490435389870957
G_est = 
 [[ 0.  0.  1.  1.]
 [ 0.  0.  0.  1.]
 [ 1.  0. -1.  0.]
 [ 0.  1.  0.  0.]]
Final Loss =  6.49843137837166
Final penalty =  2.500000026883824
Final likelihood =  3.9984313514878362
likelihood_true =  3.99747608723802


In [90]:
# Graphiacl Lasso

seed = 1

# Load dataset
n, d = 10000, 5
graph_type, degree = 'ER', 1    # ER1 graph
B_scale = 1.0
noise_type = 'gaussian_ev'

Dataset = SyntheticDataset(n, d, graph_type, degree,
                          noise_type, B_scale, seed=seed)
data = Dataset.X
B_true = Dataset.B
G_true = Dataset.B_bin

R_hat = np.cov(data.T)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)

P = glasso_problem(R_hat, n, reg_params = {'lambda1': 0.05}, latent = False, do_scaling = False)
print(P)

lambda1_range = np.logspace(0, -3, 30)
modelselect_params = {'lambda1_range': lambda1_range}

P.model_selection(modelselect_params = modelselect_params, method = 'eBIC', gamma = 0.1)

 
SINGLE GRAPHICAL LASSO PROBLEM 
Regularization parameters:
{'lambda1': 0.05, 'mu1': None}
ADMM terminated after 31 iterations with status: optimal.
ADMM terminated after 27 iterations with status: optimal.
ADMM terminated after 26 iterations with status: optimal.
ADMM terminated after 31 iterations with status: optimal.
ADMM terminated after 28 iterations with status: optimal.
ADMM terminated after 27 iterations with status: optimal.
ADMM terminated after 28 iterations with status: optimal.
ADMM terminated after 29 iterations with status: optimal.
ADMM terminated after 31 iterations with status: optimal.
ADMM terminated after 33 iterations with status: optimal.
ADMM terminated after 11 iterations with status: optimal.
ADMM terminated after 11 iterations with status: optimal.
ADMM terminated after 11 iterations with status: optimal.
ADMM terminated after 11 iterations with status: optimal.
ADMM terminated after 11 iterations with status: optimal.
ADMM terminated after 11 iterations wi

In [93]:
# Graphiacl Lasso
A_init = P.solution.adjacency_ + np.eye(d) + 0.01 * np.random.randn(*R_hat.shape)
print("A_init = \n", A_init)
beta = 5
lam = 2

A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("A_true = \n",A_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty
print("penalty_true = ", true_penalty)
print("f_true = ", f_true)

B_final,G_final, info = train_gumbel_sgd(
    Rhat_np = R_hat,
    lam = lam,
    delta = 1e-6,
    max_steps = 5000,
    tau_end = 0.05,
    B_init = A_init
)
    
print("B_est = \n",B_final)
print("G_est = \n", G_final)
print(MEC.is_in_markov_equiv_class(G_true, G_final))
print("Final Loss = ", info["final_loss"])
print("Final penalty = ", info["final_penalty"])
print("Final likelihood = ", info["final_likelihood"])
print("True likelihood = ", likelihood_true)

A_init = 
 [[1.00943208 0.97597752 0.98303461 0.99647951 0.99327032]
 [0.99550186 0.99863397 1.02242353 0.9935043  0.99304403]
 [1.0026901  1.00075488 0.99751924 1.00454864 0.98399331]
 [1.02995706 1.01125495 1.00149985 1.00192088 0.99423197]
 [1.00221175 0.99747387 0.98726606 0.97801891 0.99868381]]
A_true = 
 [[ 1.         -0.81744217  0.          0.          1.85247975]
 [ 0.          1.          0.          0.          0.        ]
 [ 0.          0.          1.          0.         -1.30384461]
 [ 0.          0.          0.          1.          1.32815971]
 [ 0.          0.          0.          0.          1.        ]]
likelihood_true =  5.007642262846862
penalty_true =  8
f_true =  13.007642262846861
First loss =  104.64835809091547
First logdet =  28.34513039727564
B_est = 
 [[ 2.09024540e+000  1.10147369e-063 -3.83044295e-065  3.64468193e-064
   7.06312584e-184]
 [-3.12602256e-068 -5.17239385e-001  1.02752222e-002 -5.20343473e-068
   7.76978378e-001]
 [-1.13433497e+000  7.28072797

In [12]:
# ER1, ground truth
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B

# Setup for logging
# Required for printing histories if checkpointing is activated
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s - %(name)s - %(message)s'
)

seed = 1

# Load dataset
seed = 1
# Load dataset
n, d = 10000, 4
graph_type, degree = 'ER', 1    # ER1 graph
B_scale = 1.0
noise_type = 'gaussian_ev'

Dataset = SyntheticDataset(n, d, graph_type, degree,
                          noise_type, B_scale, seed=seed)
data = Dataset.X
B_true = Dataset.B
G_true = Dataset.B_bin

R_hat = np.cov(data.T)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
beta = 5
lam = 0.3

A_true = np.eye(d) - B_true
# print("A_true = \n",A_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.sum(np.tanh(beta * np.abs(A_true))[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty
print("penalty_true = ", true_penalty)
print("f_true = ", f_true)

B_final,G_final, info = train_gumbel_sgd(
    Rhat_np = R_hat,
    lam = lam,
    delta = 1e-6,
    max_steps = 5000,
    tau_end = 0.05,
    B_init = A_true
)

print("A_true = \n",A_true)
print("B_est = \n",B_final)
print("G_est = \n", G_final)
print(MEC.is_in_markov_equiv_class(G_true, G_final))
print("Final Loss = ", info["final_loss"])
print("Final penalty = ", info["final_penalty"])
print("Final likelihood = ", info["final_likelihood"])
print("True likelihood = ", likelihood_true)

likelihood_true =  3.9974760872380175
penalty_true =  1.1999585829554795
f_true =  5.197434670193497
A_true = 
 [[ 1.         -1.29974793  1.78941959  1.70284777]
 [ 0.          1.          0.          0.96151608]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]
B_est = 
 [[ 1.00182887 -1.31322903  1.78955901  1.70435391]
 [ 0.          0.99817226  0.          0.96091341]
 [ 0.          0.          1.00392982  0.        ]
 [ 0.          0.          0.          0.99755366]]
G_est = 
 [[0. 1. 1. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
True
Final Loss =  5.197046865951734
Final penalty =  1.2
Final likelihood =  3.9970468659517335
True likelihood =  3.9974760872380175


In [8]:
# try random init 100 times on X -> Y <- Z
# init: random
# lam = 0.5, tau = 0.2

times = 50
i = 3
true_count = [0] * 6
for seed in range(times):
    X, Y, Z, G_true, CPDAG = generate_scm_data(i, 10000, seed = seed)
    A_true = (np.eye(3) - G_true)
    data = np.array([X, Y, Z]).T
    R_hat = np.cov(data.T)
    B_init = np.random.randn(*R_hat.shape)
    likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
    
    B_final,G_final, info = train_gumbel_sgd(
    Rhat_np = R_hat,
    lam = 1,
    delta = 1e-6,
    max_steps = 5000,
    tau_start = 0.2,
    tau_end = 0.2,
    B_init = B_init
    )
        
    print("G_est = \n",G_final)
    if MEC.is_in_markov_equiv_class(G_true, G_final): true_count[i-1] += 1
    print("Final Loss = ", info["final_loss"])
    print("Final penalty = ", info["final_penalty"])
    print("Final likelihood = ", info["final_likelihood"])
    print("likelihood_true = ", likelihood_true)
print(f"SCM {i} : {true_count[i-1]/times}")

G_est = 
 [[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -1.]]
Final Loss =  30.690671659234827
Final penalty =  7.409279589490396e-13
Final likelihood =  30.690671659234084
likelihood_true =  2.9562272795925035
G_est = 
 [[ 0.  0.  1.]
 [ 1.  0.  1.]
 [ 0.  1. -1.]]
Final Loss =  7.274970444300061
Final penalty =  4.000000000000091
Final likelihood =  3.2749704442999708
likelihood_true =  2.9921669186153315
G_est = 
 [[ 0.  0.  1.]
 [ 1.  0.  1.]
 [ 1.  0. -1.]]
Final Loss =  8.02260319328106
Final penalty =  5.00000000000003
Final likelihood =  3.0226031932810304
likelihood_true =  3.0234620384485282
G_est = 
 [[0. 1. 0.]
 [1. 0. 1.]
 [0. 1. 0.]]
Final Loss =  6.967134001814278
Final penalty =  4.000000000000021
Final likelihood =  2.967134001814257
likelihood_true =  2.9673905918882433
G_est = 
 [[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]
Final Loss =  4.963936872337479
Final penalty =  2.0000000000000977
Final likelihood =  2.963936872337381
likelihood_true =  2.964255404998236
G_est = 
 [[0. 1.

In [7]:
# init: ground truth
# lam = 0.5, tau = 0.8 -> 0.2

times = 1
for i in range(1, 6):
    true_count = [0] * 6
    for seed in range(times):
        X, Y, Z, G_true, CPDAG = generate_scm_data(i,10000, seed = seed)
        A_true = (np.eye(3) - G_true)
        data = np.array([X, Y, Z]).T
        R_hat = np.cov(data.T)
        
        #print(data.T@ data / 10000)
        B_final,G_final, info = train_gumbel_sgd(
        Rhat_np = R_hat,
        lam = 0.5,
        delta = 1e-6
    )
        
        print("G_est = \n",G_final)
        print("U = \n",info["U_final"])
        print("P = \n", info["P_final"])
        if MEC.is_in_markov_equiv_class(G_true, G_final): true_count[i-1] += 1
        print("Final Loss = ", info["final_loss"])
    print(f"SCM {i} : {true_count[i-1]/times}")

G_est = 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
U = 
 [[ 3.85615872e+01 -9.32724820e+01 -1.23261871e+02]
 [-9.68543706e+01  3.72800609e+03  2.16992312e+06]
 [-4.19412441e+00 -2.42431885e+03 -1.49890642e+03]]
P = 
 [[ 1.01254908e+00 -2.49749812e+02  2.22893109e+00]
 [-1.14750917e+02 -1.00694404e+00 -2.81226126e-25]
 [-2.30585021e-03 -9.77681917e+02 -6.32727354e+02]]
Final Loss =  30.092239038039068
SCM 1 : 1.0
G_est = 
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
U = 
 [[ 3.85663925e+01  2.51947900e+02  1.59395475e+04]
 [ 2.86208179e+01  3.72800472e+03 -1.76799532e+03]
 [-1.98398363e+03  1.43241420e+07 -1.65253935e+03]]
P = 
 [[-1.00489410e+00  1.44673812e-02  1.01254959e+00]
 [ 1.00271052e+00 -6.59696166e-03  4.56574214e+02]
 [ 3.40076880e-01  1.00699565e+00 -6.32935985e+02]]
Final Loss =  4.9557007234699
SCM 2 : 0.0
G_est = 
 [[0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
U = 
 [[  38.56560422  249.28091429   43.55168358]
 [  41.12499386 3728.00517175  391.4568214 ]
 [  23.52710143 1805.18363493  -57.1